In [ ]:
from quantumScarFunctions import *
from scipy.sparse import csr_matrix
import qutip as qt
import matplotlib.pyplot as plt
from quantumScarsPlotting import *
from scipy.optimize import curve_fit
from multiprocessing import Pool, cpu_count
import numpy as np
import time

numQubits = 19

# line reg function model
def exponential_model(x, a, b):
    return a * np.exp(b * x)

x_data = np.array(range(1, numQubits))

# flip bit hashmap
flipMap = {'0': '1', '1': '0'}

# === helper: parallel omega computation ===
def compute_for_omega(omega, matrixHamiltonian, H1, eigenstate, tlist):
    args = {"A": 0.1, "omega": omega}
    H = qt.QobjEvo([matrixHamiltonian, [H1, coeff]], args=args)
    psi_t = qt.sesolve(H, eigenstate, tlist, method="bdf")  # use sparse solver
    vals = [qt.expect(matrixHamiltonian, s) for s in psi_t.states]
    return omega, vals


wc_dict = {}  # num qubits: wc

# === main loop over system size ===
for n in range(1, numQubits):
    t0 = time.time()
    basisList = binNoConsecOnesEfficient(n)
    basisMap = {bitStr: i for i, bitStr in enumerate(basisList)}
    basisLen = len(basisList)
    flippedList = []

    row, column = [], []

    for i in range(basisLen):
        paddedBitStr = '0' + basisList[i] + '0'
        copyBit = list(paddedBitStr)

        for j in range(1, n+1):
            if paddedBitStr[j-1] == '0' and paddedBitStr[j+1] == '0':
                copyBit[j] = flipMap[paddedBitStr[j]]
                flippedList.append(''.join(copyBit)[1:-1])
                copyBit = list(paddedBitStr)

        for k in range(len(flippedList)):
            row.append(basisMap[flippedList[k]])
            column.append(i)
        flippedList.clear()

    onesList = np.ones(len(row), dtype=int)

    # === keep Hamiltonian sparse ===
    sparseHamiltonian = csr_matrix((onesList, (row, column)), shape=[basisLen, basisLen])
    matrixHamiltonian = qt.Qobj(sparseHamiltonian, dims=[[basisLen], [basisLen]])

    # diagonalize (lowest eigenstate only)
    eigenvalues, eigenstates = matrixHamiltonian.eigenstates(eigvals=1, sort='low')

    # === build H1 operator (sparse diagonal) ===
    diagH1 = []
    for i in range(basisLen):
        bitString = [int(x) for x in basisList[i]]
        z2bitString = 2 * np.array([int(x) for x in z2_initial(n)]) - 1
        diagH1.append(np.dot(2 * np.array(bitString) - 1, z2bitString))

    rowH1 = list(range(basisLen))
    columnH1 = list(range(basisLen))
    H1 = csr_matrix((diagH1, (rowH1, columnH1)), shape=[basisLen, basisLen])
    H1 = qt.Qobj(H1, dims=[[basisLen], [basisLen]])

    # === evolution settings ===
    tlist_limited = np.linspace(0, 100, 250)
    wlist_limited = np.linspace(1.0, 1.5, 100)

    # === parallel omega loop ===
    with Pool(processes=cpu_count()) as pool:
        results = pool.starmap(
            compute_for_omega,
            [(omega, matrixHamiltonian, H1, eigenstates[0], tlist_limited) for omega in wlist_limited]
        )

    expectationVals_limited = dict(results)

    # === find wc: omega with max avg energy ===
    wc_potential = {omega: max(vals) for omega, vals in expectationVals_limited.items()}
    best_omega = max(wc_potential, key=wc_potential.get)
    wc_dict[n] = float(best_omega)

    print(f"n={n} done in {time.time()-t0:.1f}s | best omega={best_omega:.4f}")

# === curve fit ===
params, covariance = curve_fit(exponential_model, x_data, list(wc_dict.values()))
a_opt, b_opt = params
y_predicted = exponential_model(x_data, a_opt, b_opt)

plt.figure()
for key, value in wc_dict.items():
    plt.plot(key, value, ".")
plt.plot(x_data, y_predicted)
plt.xlabel("Num Qubits")
plt.ylabel("wc")
plt.title("wc vs. Num Qubits")
plt.show()

print("Fit params:", a_opt, b_opt)